In [1]:
from IPython.display import display, Markdown
import os
import json
from libcbm import resources
import pandas as pd
import numpy as np
from numpy.random import default_rng
from libcbm.model.cbm_exn import cbm_exn_model
from libcbm.model.cbm_exn import cbm_exn_spinup
from libcbm.model.cbm_exn import cbm_exn_step
from libcbm.model.cbm_exn.cbm_exn_parameters import parameters_factory
from libcbm.model.model_definition.model_variables import ModelVariables

In [2]:
rng = default_rng()
n_stands = 10

In [3]:
# load the default bundled parameters
parameters = parameters_factory(resources.get_cbm_exn_parameters_dir())

# Fully dynamic modelling with cbm_exn

This document illustrates a scheme to simulate with fully dynamic pools, fluxes and flow in libcbm using the cbm_exn package.  

This outlines a component of a highly efficient method for processing C flows in a CBM-Like model, producing outputs that are compatible with current reporting methods, and systems.

The approach makes the hard coded model pool, flow, flux in CBM-CFS3 fully dynamic, and definable via high level language in the cbm_exn package.  

Prior to this improvement, using CBM-CFS3, it was only possible to modify parameters for the pool-flows withing the hard-coded model structure.  This opens up opportunities to evaluate changes to model structure dynamically, in addition to modifiying pool flows.  

In addition the architecture opens opportunities for using machine learning both for informing pool flow parameters and the model structure itself.

The document explains the basic structure of the default parameters and configuration in `cbm_exn`, and also shows the an example of the default pool flows generated in dataframe form, and describes their format. 

While only the default pool flows (based on CBM-CFS3) are presented here, it's important to note if the general format is followed it's possible to alter model pool-flow-flux structure via high level programming language.  The default values could potentially be used as a template to accomplish this.

Sections below are:

 * Operation DataFrame: a description of the storage scheme for mult-dimensional pool flow proportions within dataframe structures.
 * Default spinup operation dataframes: examples of spinup ops derived from the cbm_exn default parameters and example simulation area input
 * Default step operation dataframes: example of step ops also derived from the same inputs
 * Appendix1, Appendix2: the default parameters, and example simulation area used.


In [4]:
# read some packaged net increments, derived from a
# simulation of the same growth curve used in CBM-CFS3
# tutorial 1
net_increments = pd.read_csv(
    os.path.join(
        resources.get_test_resources_dir(),
        "cbm_exn_net_increments",
        "net_increments.csv"
    )
)

In [5]:
# the same set of increments are repeated for each stand in this example
n_stands = 1000
increments = None
for s in range(n_stands):
    s_increments = net_increments.copy()
    s_increments.insert(0, "row_idx", s)
    s_increments = s_increments.rename(
        columns={
            "SoftwoodMerch": "merch_inc",
            "SoftwoodFoliage": "foliage_inc",
            "SoftwoodOther": "other_inc"
    })
    increments = pd.concat([increments, s_increments])

In [6]:
# create the require inputs for spinup
spinup_input = {
    "parameters": pd.DataFrame(
        {
            # random age
            "age": rng.integers(low=0, high=60, size=n_stands, dtype="int"),
            "area": np.full(n_stands, 1, dtype="int"),
            "delay": np.full(n_stands, 0, dtype="int"),
            "return_interval": np.full(n_stands, 125, dtype="int"),
            "min_rotations": np.full(n_stands, 10, dtype="int"),
            "max_rotations": np.full(n_stands, 30, dtype="int"),
            "spatial_unit_id": np.full(n_stands, 17, dtype="int"), # ontario/mixedwood plains
            "species": np.full(n_stands, 20, dtype="int"), # red pine
            "mean_annual_temperature":  np.arange(0, 2, 2/n_stands), # make a temperature ramp
            "historical_disturbance_type": np.full(n_stands, 1, dtype="int"),
            "last_pass_disturbance_type": np.full(n_stands, 1, dtype="int"),
        }
    ),
    "increments":increments,
}

In [7]:
spinup_vars = cbm_exn_spinup.prepare_spinup_vars(
    ModelVariables.from_pandas(spinup_input),
    parameters,
)
spinup_op_list = cbm_exn_spinup.get_default_op_list()
spinup_ops = cbm_exn_spinup.get_default_ops(parameters, spinup_vars)

In [8]:
spinup_op_list

['growth',
 'snag_turnover',
 'biomass_turnover',
 'overmature_decline',
 'growth',
 'dom_decay',
 'slow_decay',
 'slow_mixing',
 'disturbance']

# Operation dataframes

Operation dataframes are multi-dimensional sparse matrices, using column-name-formatting to denote the dimensions.  

The rationale for this design apporach is that multi-dimensional arrays lack interoperable standards that would work with many different high level languages simulataneously.  Dataframes however are well supported by several high level languages.

Each dataframe has pool flow columns, where the column name is of the form:

    pool_source_name.pool_sink_name
    
Each row can be interpreted as a pool flow matrix (in sparse coordinate form).  By default unspecified diagnal values (those where `pool_source_name` is equal to `pool_sink_name`) will be assigned a value of 1.

There are 4 basic types of dataframes with regards to mapping to simulation space.  These are covered in the next section

## Single row 

The single row is a matrix that is applied to all simulation areas.  See the `slow_mixing` dataframe in the following section.

## Simulation-aligned 

the dataframe has a row for each simulation area in subsequent spinup or step calls.  Each row represents a matrix that is 1:1 with simulation areas
This type is appropriate for processes that vary by simulation area. See the `dom_decay` dataframe in the followin section.

## Property-indexed 

A property-index dataframe has one or more values that correspond to values stored in the current simulation state. This type of dataframe contains 1 or more columns of the form:

    [table_name.variable_name]

Where a simulation table, series name pair is surrounded by left and right brackets.

An example of this is if one flow matrix is defined for each spatial unit identifier.  See the `disturbance` dataframe in the following section for an example.

## Both Simulation-aligned, and Property-indexed

If a dataframe is both simulation aligned and property indexed each row corresponds to both a simulation area, and 1 or more properties within the simulation areas.  The following pattern of columns is present in this type of dataframe:

    [row_idx], [table_name_1.variable_name_1], ... [table_name_N.variable_name_N]

An example of this is a dataframe of matrices where each row corresponds to the simulation areas, and to simulation age.  See the spinup growth dataframe in the following section.

# Default spinup operation dataframes
The default spinup operation dataframes are generated as a function of the default parameters (see appendix 1) and the simulation area input (Appendix 2).  

The [libcbm.model.cbm_exn.cbm_exn_spinup.spinup](https://github.com/cat-cfs/libcbm_py/blob/700b2febb73681ca2b4456d4db88d5c399008640/libcbm/model/cbm_exn/cbm_exn_spinup.py#L169) function can directly ingest operations in this format via the `ops` parameter.

Within a spinup timestep these operations are applied in the following order by default.  The order and naming of these operation is user-specifyable via passing a string list to the `op_sequence` parameter of the above linked function

 1. growth
 1. snag_turnover
 1. biomass_turnover
 1. overmature_decline
 1. growth
 1. dom_decay
 1. slow_decay
 1. slow_mixing
 1. disturbance

In [10]:
for op in spinup_ops:
    display(Markdown(f"## {op['name']}"))
    display(op["op_data"])

## snag_turnover

,[parameters.spatial_unit_id],[parameters.sw_hw],StemSnag.StemSnag,StemSnag.MediumSoil,BranchSnag.BranchSnag,BranchSnag.AboveGroundFastSoil
0,1,0,0.968,0.032,0.9,0.1
1,3,0,0.968,0.032,0.9,0.1
2,4,0,0.968,0.032,0.9,0.1
3,5,0,0.968,0.032,0.9,0.1
4,6,0,0.968,0.032,0.9,0.1
...,...,...,...,...,...,...
91,52,1,0.968,0.032,0.9,0.1
92,53,1,0.968,0.032,0.9,0.1
93,54,1,0.968,0.032,0.9,0.1
94,58,1,0.968,0.032,0.9,0.1


## biomass_turnover

,[parameters.spatial_unit_id],[parameters.sw_hw],Merch.StemSnag,Foliage.AboveGroundVeryFastSoil,Other.BranchSnag,Other.AboveGroundFastSoil,CoarseRoots.AboveGroundFastSoil,CoarseRoots.BelowGroundFastSoil,FineRoots.AboveGroundVeryFastSoil,FineRoots.BelowGroundVeryFastSoil
0,1,0,0.0050,0.10,0.0100,0.0300,0.01,0.01,0.3205,0.3205
1,3,0,0.0060,0.05,0.0075,0.0225,0.01,0.01,0.3205,0.3205
2,4,0,0.0050,0.10,0.0100,0.0300,0.01,0.01,0.3205,0.3205
3,5,0,0.0067,0.15,0.0100,0.0300,0.01,0.01,0.3205,0.3205
4,6,0,0.0067,0.15,0.0100,0.0300,0.01,0.01,0.3205,0.3205
...,...,...,...,...,...,...,...,...,...,...
91,52,1,0.0050,0.95,0.0100,0.0300,0.01,0.01,0.3205,0.3205
92,53,1,0.0060,0.95,0.0100,0.0300,0.01,0.01,0.3205,0.3205
93,54,1,0.0045,0.95,0.0100,0.0300,0.01,0.01,0.3205,0.3205
94,58,1,0.0060,0.95,0.0075,0.0225,0.01,0.01,0.3205,0.3205


## dom_decay

...

,[parameters.spatial_unit_id],[state.disturbance_type],[parameters.sw_hw],AboveGroundFastSoil.AboveGroundFastSoil,AboveGroundFastSoil.AboveGroundSlowSoil,AboveGroundFastSoil.BelowGroundFastSoil,AboveGroundFastSoil.CH4,AboveGroundFastSoil.CO,AboveGroundFastSoil.CO2,AboveGroundFastSoil.MediumSoil,...,Other.Other,Other.Products,Products.Products,StemSnag.AboveGroundSlowSoil,StemSnag.CH4,StemSnag.CO,StemSnag.CO2,StemSnag.MediumSoil,StemSnag.Products,StemSnag.StemSnag
0,1,0,0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3,0,0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4,0,0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5,0,0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,6,0,0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12747,52,1,1,0.386657,0.0,0.0,0.006133,0.055201,0.552009,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12748,53,1,1,0.444126,0.0,0.0,0.005559,0.050029,0.500287,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12749,54,1,1,0.448397,0.0,0.0,0.005516,0.049644,0.496443,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12750,58,1,1,0.338494,0.0,0.0,0.006615,0.059536,0.595355,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## growth

,[row_idx],[state.age],Input.Merch,Input.Other,Input.Foliage,Input.CoarseRoots,Input.FineRoots
0,0,0,0.009449,0.009383,0.020006,0.004955,0.003667
1,0,1,0.041615,0.049833,0.040492,0.016929,0.012362
2,0,2,0.085850,0.094331,0.054661,0.030487,0.021648
3,0,3,0.138499,0.138077,0.066074,0.045285,0.030783
4,0,4,0.197727,0.179735,0.075630,0.061304,0.039282
...,...,...,...,...,...,...,...
70995,999,66,0.000000,0.000000,0.000000,0.000000,0.000000
70996,999,67,0.000000,0.000000,0.000000,0.000000,0.000000
70997,999,68,0.000000,0.000000,0.000000,0.000000,0.000000
70998,999,69,0.000000,0.000000,0.000000,0.000000,0.000000


## overmature_decline

,[row_idx],[state.age],Merch.StemSnag,Other.BranchSnag,Other.AboveGroundFastSoil,Foliage.AboveGroundVeryFastSoil,CoarseRoots.AboveGroundFastSoil,CoarseRoots.BelowGroundFastSoil,FineRoots.AboveGroundVeryFastSoil,FineRoots.BelowGroundVeryFastSoil
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
70995,999,66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70996,999,67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70997,999,68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70998,999,69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Run the spinup routine with the default operations passed as a parameter


In [11]:
with cbm_exn_model.initialize() as model:
    cbm_vars = cbm_exn_spinup.spinup(
        model,
        spinup_vars,
        ops=spinup_ops,
        op_sequence=spinup_op_list
    )

In [19]:
step_ops_sequence = cbm_exn_step.get_default_annual_process_op_sequence()
step_disturbance_ops_sequence = cbm_exn_step.get_default_disturbance_op_sequence()
step_ops = cbm_exn_step.get_default_ops(parameters, cbm_vars)

In [24]:
step_disturbance_ops_sequence

['disturbance']

In [23]:
step_ops_sequence

['growth',
 'snag_turnover',
 'biomass_turnover',
 'overmature_decline',
 'growth',
 'dom_decay',
 'slow_decay',
 'slow_mixing']

# Step processes

With the exception of growth and overmature decline, the default step processes use an identical process for generation as the spinup processes, and so only growth and overmature decline are shown below.

The difference is that by default in stepping, growth and overmature decline dataframes are both Simulation-aligned.


The [libcbm.model.cbm_exn.cbm_exn_step.step](https://github.com/cat-cfs/libcbm_py/blob/700b2febb73681ca2b4456d4db88d5c399008640/libcbm/model/cbm_exn/cbm_exn_step.py#L190) function can directly ingest operations in this format via the `ops` parameter

In [21]:
for op in step_ops:
    name = op['name']
    if name in ["growth", "overmature_decline"]:
        display(Markdown(f"## {name}"))
        display(op["op_data"])

## growth

,Input.Merch,Input.Other,Input.Foliage,Input.CoarseRoots,Input.FineRoots
0,0.05,0.025,0.005,0.014796,0.002964
1,0.05,0.025,0.005,0.017125,0.000635
2,0.05,0.025,0.005,0.017158,0.000602
3,0.05,0.025,0.005,0.016998,0.000762
4,0.05,0.025,0.005,0.017292,0.000468
...,...,...,...,...,...
995,0.05,0.025,0.005,0.016136,0.001624
996,0.05,0.025,0.005,0.011324,0.006436
997,0.05,0.025,0.005,0.017037,0.000723
998,0.05,0.025,0.005,0.012568,0.005192


## overmature_decline

,Merch.StemSnag,Other.BranchSnag,Other.AboveGroundFastSoil,Foliage.AboveGroundVeryFastSoil,CoarseRoots.AboveGroundFastSoil,CoarseRoots.BelowGroundFastSoil,FineRoots.AboveGroundVeryFastSoil,FineRoots.BelowGroundVeryFastSoil
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# initialize parameters for stepping (values for illustration)
cbm_vars["parameters"]["mean_annual_temperature"].assign(1.1)
cbm_vars["parameters"]["merch_inc"].assign(0.1)
cbm_vars["parameters"]["foliage_inc"].assign(0.01)
cbm_vars["parameters"]["other_inc"].assign(0.05)
cbm_vars["parameters"]["disturbance_type"].assign(
    rng.choice(
        [0,1,4], n_stands, p=[0.98, 0.01, 0.01]
    )
)

run a timestep with the specified parameters

In [25]:
with cbm_exn_model.initialize() as model:
    cbm_vars = cbm_exn_step.step(
        model,
        cbm_vars,
        ops=step_ops,
        step_op_sequence=step_ops_sequence,
        disturbance_op_sequence=step_disturbance_ops_sequence
    )

In [27]:
cbm_vars["pools"].to_pandas()

,Input,Merch,Foliage,Other,CoarseRoots,FineRoots,AboveGroundVeryFastSoil,BelowGroundVeryFastSoil,AboveGroundFastSoil,BelowGroundFastSoil,MediumSoil,AboveGroundSlowSoil,BelowGroundSlowSoil,StemSnag,BranchSnag,CO2,CH4,CO,NO2,Products
0,1.0,11.051449,2.377428,6.848106,3.250694,1.250797,2.830523,0.940860,13.273371,5.458214,60.235588,34.429093,129.466330,49.808278,2.497954,7616.639019,8.475443,76.277752,0.0,0.0
1,1.0,79.877452,7.727531,22.616777,22.252566,2.216664,10.721424,2.032651,11.240203,2.540329,56.134844,39.639266,127.581073,20.682872,1.466552,7741.999074,8.474532,76.269556,0.0,0.0
2,1.0,0.100000,0.010000,0.050000,0.034317,0.001203,0.855133,2.287202,15.142328,12.140026,68.723296,22.825277,127.696620,73.470779,13.649406,7787.539497,9.073830,81.663143,0.0,0.0
3,1.0,91.075774,8.803881,25.692223,25.526021,2.350936,12.556958,2.179480,13.486569,2.930162,50.902928,42.737867,127.414759,17.952714,1.761883,7804.580054,8.472711,76.253165,0.0,0.0
4,1.0,62.828501,6.371858,18.489765,17.404597,2.062611,8.861246,1.881570,9.914532,2.416727,59.473614,37.781043,127.935909,24.660480,1.217290,7701.972907,8.471801,76.244972,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,19.261305,3.303527,10.027807,5.642938,1.592627,3.795571,1.145445,9.812443,3.377508,55.151745,33.141562,124.520688,41.501841,1.581012,7658.169537,7.617916,68.560102,0.0,0.0
996,1.0,1.570764,0.691218,1.429916,0.497720,0.321881,0.765333,0.446949,16.326757,8.779878,47.968592,31.162649,125.554447,68.631818,6.771527,7609.015153,7.617101,68.552767,0.0,0.0
997,1.0,87.624403,8.470265,24.719428,24.514003,2.306726,9.962810,1.858328,10.991149,2.386592,45.536334,39.088129,122.897160,18.083168,1.595287,7803.115806,7.616286,68.545434,0.0,0.0
998,1.0,3.993677,1.268554,3.139598,1.203462,0.661744,1.237934,0.474181,14.339180,6.808633,50.982204,31.747401,125.304401,60.331773,4.420749,7621.906900,7.615472,68.538101,0.0,0.0


In [28]:
cbm_vars["flux"].to_pandas()

,DisturbanceCO2Production,DisturbanceCH4Production,DisturbanceCOProduction,DisturbanceBioCO2Emission,DisturbanceBioCH4Emission,DisturbanceBioCOEmission,DecayDOMCO2Emission,DisturbanceProduction,DisturbanceDOMProduction,DeltaBiomass_AG,...,DisturbanceFineLitterInput,DisturbanceVFastAGToAir,DisturbanceVFastBGToAir,DisturbanceFastAGToAir,DisturbanceFastBGToAir,DisturbanceMediumToAir,DisturbanceSlowAGToAir,DisturbanceSlowBGToAir,DisturbanceStemSnagToAir,DisturbanceBranchSnagToAir
0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,4.180065,0.0,0.0,0.16,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,4.972953,0.0,0.0,0.16,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
2,54.017735,0.600208,5.401783,12.32667,0.136972,1.23266,5.544954,0.0,0.0,0.16,...,1.740541,10.282922,0.0,8.021325,0.0,11.473104,16.546073,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,5.380294,0.0,0.0,0.16,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,4.631717,0.0,0.0,0.16,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,3.782185,0.0,0.0,0.16,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
996,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,4.236419,0.0,0.0,0.16,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
997,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,4.565181,0.0,0.0,0.16,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
998,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,3.942082,0.0,0.0,0.16,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0


## Appendix 1: CBM EXN Default parameters

In [16]:

for k, v in parameters._data.items():
    display(Markdown(f"## {k}"))
    if isinstance(v, list):
        display(Markdown(f"```json\n{json.dumps(v, indent=4)}\n```"))
    else:
        display(v)

## pools

```json
[
    "Input",
    "Merch",
    "Foliage",
    "Other",
    "CoarseRoots",
    "FineRoots",
    "AboveGroundVeryFastSoil",
    "BelowGroundVeryFastSoil",
    "AboveGroundFastSoil",
    "BelowGroundFastSoil",
    "MediumSoil",
    "AboveGroundSlowSoil",
    "BelowGroundSlowSoil",
    "StemSnag",
    "BranchSnag",
    "CO2",
    "CH4",
    "CO",
    "NO2",
    "Products"
]
```

## flux

```json
[
    {
        "name": "DisturbanceCO2Production",
        "process": "Disturbance",
        "source_pools": [
            "Merch",
            "Foliage",
            "Other",
            "CoarseRoots",
            "FineRoots",
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "MediumSoil",
            "AboveGroundSlowSoil",
            "BelowGroundSlowSoil",
            "StemSnag",
            "BranchSnag"
        ],
        "sink_pools": [
            "CO2"
        ]
    },
    {
        "name": "DisturbanceCH4Production",
        "process": "Disturbance",
        "source_pools": [
            "Merch",
            "Foliage",
            "Other",
            "CoarseRoots",
            "FineRoots",
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "MediumSoil",
            "AboveGroundSlowSoil",
            "BelowGroundSlowSoil",
            "StemSnag",
            "BranchSnag"
        ],
        "sink_pools": [
            "CH4"
        ]
    },
    {
        "name": "DisturbanceCOProduction",
        "process": "Disturbance",
        "source_pools": [
            "Merch",
            "Foliage",
            "Other",
            "CoarseRoots",
            "FineRoots",
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "MediumSoil",
            "AboveGroundSlowSoil",
            "BelowGroundSlowSoil",
            "StemSnag",
            "BranchSnag"
        ],
        "sink_pools": [
            "CO"
        ]
    },
    {
        "name": "DisturbanceBioCO2Emission",
        "process": "Disturbance",
        "source_pools": [
            "Merch",
            "Foliage",
            "Other",
            "CoarseRoots",
            "FineRoots"
        ],
        "sink_pools": [
            "CO2"
        ]
    },
    {
        "name": "DisturbanceBioCH4Emission",
        "process": "Disturbance",
        "source_pools": [
            "Merch",
            "Foliage",
            "Other",
            "CoarseRoots",
            "FineRoots"
        ],
        "sink_pools": [
            "CH4"
        ]
    },
    {
        "name": "DisturbanceBioCOEmission",
        "process": "Disturbance",
        "source_pools": [
            "Merch",
            "Foliage",
            "Other",
            "CoarseRoots",
            "FineRoots"
        ],
        "sink_pools": [
            "CO"
        ]
    },
    {
        "name": "DecayDOMCO2Emission",
        "process": "Decay",
        "source_pools": [
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "MediumSoil",
            "AboveGroundSlowSoil",
            "BelowGroundSlowSoil",
            "StemSnag",
            "BranchSnag"
        ],
        "sink_pools": [
            "CO2"
        ]
    },
    {
        "name": "DisturbanceProduction",
        "process": "Disturbance",
        "source_pools": [
            "Merch",
            "Foliage",
            "Other",
            "CoarseRoots",
            "FineRoots"
        ],
        "sink_pools": [
            "Products"
        ]
    },
    {
        "name": "DisturbanceDOMProduction",
        "process": "Disturbance",
        "source_pools": [
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "MediumSoil",
            "AboveGroundSlowSoil",
            "BelowGroundSlowSoil",
            "StemSnag",
            "BranchSnag"
        ],
        "sink_pools": [
            "Products"
        ]
    },
    {
        "name": "DeltaBiomass_AG",
        "process": "Growth and Turnover",
        "source_pools": [
            "Input"
        ],
        "sink_pools": [
            "Merch",
            "Foliage",
            "Other"
        ]
    },
    {
        "name": "DeltaBiomass_BG",
        "process": "Growth and Turnover",
        "source_pools": [
            "Input"
        ],
        "sink_pools": [
            "CoarseRoots",
            "FineRoots"
        ]
    },
    {
        "name": "TurnoverMerchLitterInput",
        "process": "Growth and Turnover",
        "source_pools": [
            "Merch"
        ],
        "sink_pools": [
            "StemSnag"
        ]
    },
    {
        "name": "TurnoverFolLitterInput",
        "process": "Growth and Turnover",
        "source_pools": [
            "Foliage"
        ],
        "sink_pools": [
            "AboveGroundVeryFastSoil"
        ]
    },
    {
        "name": "TurnoverOthLitterInput",
        "process": "Growth and Turnover",
        "source_pools": [
            "Other"
        ],
        "sink_pools": [
            "AboveGroundFastSoil",
            "BranchSnag"
        ]
    },
    {
        "name": "TurnoverCoarseLitterInput",
        "process": "Growth and Turnover",
        "source_pools": [
            "CoarseRoots"
        ],
        "sink_pools": [
            "AboveGroundFastSoil",
            "BelowGroundFastSoil"
        ]
    },
    {
        "name": "TurnoverFineLitterInput",
        "process": "Growth and Turnover",
        "source_pools": [
            "FineRoots"
        ],
        "sink_pools": [
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil"
        ]
    },
    {
        "name": "DecayVFastAGToAir",
        "process": "Decay",
        "source_pools": [
            "AboveGroundVeryFastSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DecayVFastBGToAir",
        "process": "Decay",
        "source_pools": [
            "BelowGroundVeryFastSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DecayFastAGToAir",
        "process": "Decay",
        "source_pools": [
            "AboveGroundFastSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DecayFastBGToAir",
        "process": "Decay",
        "source_pools": [
            "BelowGroundFastSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DecayMediumToAir",
        "process": "Decay",
        "source_pools": [
            "MediumSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DecaySlowAGToAir",
        "process": "Decay",
        "source_pools": [
            "AboveGroundSlowSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DecaySlowBGToAir",
        "process": "Decay",
        "source_pools": [
            "BelowGroundSlowSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DecayStemSnagToAir",
        "process": "Decay",
        "source_pools": [
            "StemSnag"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DecayBranchSnagToAir",
        "process": "Decay",
        "source_pools": [
            "BranchSnag"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceMerchToAir",
        "process": "Disturbance",
        "source_pools": [
            "Merch"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceFolToAir",
        "process": "Disturbance",
        "source_pools": [
            "Foliage"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceOthToAir",
        "process": "Disturbance",
        "source_pools": [
            "Other"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceCoarseToAir",
        "process": "Disturbance",
        "source_pools": [
            "CoarseRoots"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceFineToAir",
        "process": "Disturbance",
        "source_pools": [
            "FineRoots"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceDOMCO2Emission",
        "process": "Disturbance",
        "source_pools": [
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "MediumSoil",
            "AboveGroundSlowSoil",
            "BelowGroundSlowSoil",
            "StemSnag",
            "BranchSnag"
        ],
        "sink_pools": [
            "CO2"
        ]
    },
    {
        "name": "DisturbanceDOMCH4Emission",
        "process": "Disturbance",
        "source_pools": [
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "MediumSoil",
            "AboveGroundSlowSoil",
            "BelowGroundSlowSoil",
            "StemSnag",
            "BranchSnag"
        ],
        "sink_pools": [
            "CH4"
        ]
    },
    {
        "name": "DisturbanceDOMCOEmission",
        "process": "Disturbance",
        "source_pools": [
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "MediumSoil",
            "AboveGroundSlowSoil",
            "BelowGroundSlowSoil",
            "StemSnag",
            "BranchSnag"
        ],
        "sink_pools": [
            "CO"
        ]
    },
    {
        "name": "DisturbanceMerchLitterInput",
        "process": "Disturbance",
        "source_pools": [
            "Merch"
        ],
        "sink_pools": [
            "StemSnag",
            "MediumSoil",
            "AboveGroundSlowSoil"
        ]
    },
    {
        "name": "DisturbanceFolLitterInput",
        "process": "Disturbance",
        "source_pools": [
            "Foliage"
        ],
        "sink_pools": [
            "AboveGroundVeryFastSoil",
            "BelowGroundVeryFastSoil"
        ]
    },
    {
        "name": "DisturbanceOthLitterInput",
        "process": "Disturbance",
        "source_pools": [
            "Other"
        ],
        "sink_pools": [
            "AboveGroundFastSoil",
            "BranchSnag",
            "BelowGroundFastSoil"
        ]
    },
    {
        "name": "DisturbanceCoarseLitterInput",
        "process": "Disturbance",
        "source_pools": [
            "CoarseRoots"
        ],
        "sink_pools": [
            "AboveGroundFastSoil",
            "BelowGroundFastSoil"
        ]
    },
    {
        "name": "DisturbanceFineLitterInput",
        "process": "Disturbance",
        "source_pools": [
            "FineRoots"
        ],
        "sink_pools": [
            "AboveGroundVeryFastSoil",
            "AboveGroundFastSoil",
            "BelowGroundFastSoil",
            "BelowGroundVeryFastSoil"
        ]
    },
    {
        "name": "DisturbanceVFastAGToAir",
        "process": "Disturbance",
        "source_pools": [
            "AboveGroundVeryFastSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceVFastBGToAir",
        "process": "Disturbance",
        "source_pools": [
            "BelowGroundVeryFastSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceFastAGToAir",
        "process": "Disturbance",
        "source_pools": [
            "AboveGroundFastSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceFastBGToAir",
        "process": "Disturbance",
        "source_pools": [
            "BelowGroundFastSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceMediumToAir",
        "process": "Disturbance",
        "source_pools": [
            "MediumSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceSlowAGToAir",
        "process": "Disturbance",
        "source_pools": [
            "AboveGroundSlowSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceSlowBGToAir",
        "process": "Disturbance",
        "source_pools": [
            "BelowGroundSlowSoil"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceStemSnagToAir",
        "process": "Disturbance",
        "source_pools": [
            "StemSnag"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    },
    {
        "name": "DisturbanceBranchSnagToAir",
        "process": "Disturbance",
        "source_pools": [
            "BranchSnag"
        ],
        "sink_pools": [
            "CO2",
            "CH4",
            "CO",
            "NO2"
        ]
    }
]
```

## slow_mixing_rate

...

,pool,base_decay_rate,reference_temp,q10,prop_to_atmosphere,max_rate
0,AboveGroundVeryFastSoil,0.35500,10.0,2.65,0.815,1.0
1,BelowGroundVeryFastSoil,0.50000,10.0,2.00,0.830,1.0
2,AboveGroundFastSoil,0.14350,10.0,2.00,0.830,1.0
3,BelowGroundFastSoil,0.14350,10.0,2.00,0.830,1.0
4,MediumSoil,0.03740,10.0,2.00,0.830,1.0
5,AboveGroundSlowSoil,0.01500,10.0,2.65,1.000,1.0
6,BelowGroundSlowSoil,0.00330,10.0,1.00,1.000,1.0
7,StemSnag,0.01870,10.0,2.00,0.830,1.0
8,BranchSnag,0.07175,10.0,2.00,0.830,1.0


## disturbance_matrix_value

,disturbance_matrix_id,source_pool,sink_pool,proportion
0,2,Merch,StemSnag,0.68624
1,2,Merch,CO2,0.28102
2,2,Merch,CH4,0.00316
3,2,Merch,CO,0.02842
4,2,Foliage,CO2,0.85570
...,...,...,...,...
14788,960,MediumSoil,MediumSoil,1.00000
14789,960,AboveGroundSlowSoil,AboveGroundSlowSoil,1.00000
14790,960,BelowGroundSlowSoil,BelowGroundSlowSoil,1.00000
14791,960,StemSnag,StemSnag,1.00000


## disturbance_matrix_association

,spatial_unit_id,disturbance_type_id,sw_hw,disturbance_matrix_id
0,1,0,0,27
1,3,0,0,27
2,4,0,0,27
3,5,0,0,27
4,6,0,0,27
...,...,...,...,...
12747,52,1,1,850
12748,53,1,1,862
12749,54,1,1,845
12750,58,1,1,888


## Appendix 2: simulation parameters used to generate op-dataframes

### Net C increments

These are derived from Tutorial 1 of the CBM-CFS3 Operational-Scale toolbox.  Each simulation area is assigned the same set of increments.

In [ ]:
display(increments)

### Simulation areas

In [ ]:
display(spinup_input["parameters"])